## Музеи Санкт-Петербурга: сбор информациипо API c Яндекс карт

Получаю ключ доступа в [API Яндекс.Картах](https://yandex.ru/dev/maps/), затем делаю запрос, чтобы утащить понравившуюся информацию о музеях.

In [125]:
import requests as rq
import pandas as pd
import json as k
r = rq.get('https://search-maps.yandex.ru/v1/?text=музеи санкт-петербург&bbox=59.816330,30.189180~60.018651,30.479026&results=500&lang=ru_RU&&apikey=b6a35fa9-79d2-477c-a8ae-f679ea4b115e')

Смотрю, как выглядит информация об 1 объекте:

In [134]:
k.loads(r.text)['features'][0]

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [30.332383, 59.938782]},
 'properties': {'name': 'Русский музей, Михайловский дворец',
  'description': 'Инженерная ул., 2-4А, Санкт-Петербург, Россия',
  'boundedBy': [[30.3282775, 59.93672194], [30.3364885, 59.94084194]],
  'CompanyMetaData': {'id': '1094851452',
   'name': 'Русский музей, Михайловский дворец',
   'address': 'Россия, Санкт-Петербург, Инженерная улица, 2-4А',
   'url': 'http://rusmuseum.ru/',
   'Phones': [{'type': 'phone', 'formatted': '+7 (812) 314-34-48'},
    {'type': 'phone', 'formatted': '+7 (812) 595-42-48'},
    {'type': 'phone', 'formatted': '+7 (812) 570-56-91'},
    {'type': 'phone', 'formatted': '+7 (812) 314-83-68'}],
   'Categories': [{'class': 'museum', 'name': 'Музей'}],
   'Hours': {'text': 'пн 10:00–19:30; ср 10:00–17:30; чт 13:00–20:30; пт-вс 10:00–17:30',
    'Availabilities': [{'Intervals': [{'from': '10:00:00', 'to': '19:30:00'}],
      'Monday': True},
     {'Intervals': [{'from':

Цикл, который собирает координаты, название, адрес и id:

In [135]:
coord = []
name = []
adress = []
id = []

for i in range(0,len(k.loads(r.text)['features'])):
    coord.append(k.loads(r.text)['features'][i]["geometry"]['coordinates'])
    name.append(k.loads(r.text)['features'][i]['properties']['name'])
    adress.append(k.loads(r.text)['features'][i]['properties']['description'])
    id.append(k.loads(r.text)['features'][i]['properties']['CompanyMetaData']['id'])
    i = i+1

Отдельно собираю ссылки на сайты, телефоны, категории и режим работы, потому что есть пропущенные значения:

In [137]:
url = []
for i in range(0,len(k.loads(r.text)['features'])):
    if 'url' in k.loads(r.text)['features'][i]['properties']['CompanyMetaData']:
        url.append(k.loads(r.text)['features'][i]['properties']['CompanyMetaData']['url'])
    else:
        url.append('-')
    i=i+1

In [138]:
phone = []
for i in range(0,len(k.loads(r.text)['features'])):
    if 'Phones' in k.loads(r.text)['features'][i]['properties']['CompanyMetaData']:
        phone.append(k.loads(r.text)['features'][i]['properties']['CompanyMetaData']['Phones'][0]['formatted'])
    else:
        phone.append('-')
    i=i+1

In [139]:
category1 = []
for i in range(0,len(k.loads(r.text)['features'])):
    if 'Categories' in k.loads(r.text)['features'][i]['properties']['CompanyMetaData'] and 'name' in k.loads(r.text)['features'][i]['properties']['CompanyMetaData']['Categories'][0]:
        category1.append(k.loads(r.text)['features'][i]['properties']['CompanyMetaData']['Categories'][0]['name'])
    else:
        category1.append('-')
    i=i+1

In [140]:
category2 = []
for i in range(0,len(k.loads(r.text)['features'])):
    if 'Categories' in k.loads(r.text)['features'][i]['properties']['CompanyMetaData'] and len(k.loads(r.text)['features'][i]['properties']['CompanyMetaData']['Categories'])==2:
        category2.append(k.loads(r.text)['features'][i]['properties']['CompanyMetaData']['Categories'][1]['name'])
    else:
        category2.append('-')
    i=i+1

In [141]:
hours=[]
for i in range(0,len(k.loads(r.text)['features'])):
    if 'Hours' in k.loads(r.text)['features'][i]['properties']['CompanyMetaData']:
        hours.append(k.loads(r.text)['features'][i]['properties']['CompanyMetaData']['Hours']['text'])
    else:
        hours.append('-')
    i=i+1

In [147]:
data = pd.DataFrame()
data['coord']=coord
data['name']=name
data['adress']=adress
data['id']=id
data['url']=url
data['phone']=phone
data['category1']=category1
data['category2']=category2
data['hours']=hours

Данные собраны и записаны в Excel:

In [148]:
data
data.to_excel('kudago_data.xlsx', sheet_name='Sheet1')

,coord,name,adress,id,url,phone,category1,category2,hours
0,"[30.332383, 59.938782]","Русский музей, Михайловский дворец","Инженерная ул., 2-4А, Санкт-Петербург, Россия",1094851452,http://rusmuseum.ru/,+7 (812) 314-34-48,Музей,-,пн 10:00–19:30; ср 10:00–17:30; чт 13:00–20:30...
1,"[30.330375, 59.887984]",Гранд Макет Россия,"Россия, Санкт-Петербург, Цветочная улица, 16Л",1118815278,https://grandmaket.ru/,+7 (812) 495-54-65,Музей,-,"ежедневно, 10:00–19:15"
2,"[30.343181, 59.934674]",Музей Фаберже,"наб. реки Фонтанки, 21, Санкт-Петербург, Россия",1181267939,http://fabergemuseum.ru/,+7 (812) 333-26-55,Музей,-,"ежедневно, 09:30–20:15"
3,"[30.314566, 59.939864]",Государственный Эрмитаж,"Дворцовая наб., 34, Санкт-Петербург, Россия",1057721048,http://www.hermitagemuseum.org/,+7 (812) 571-34-65,Музей,Достопримечательность,вт-пт 11:00–18:00; сб 11:00–20:00; вс 11:00–18:00
4,"[30.29347, 59.929722]",Центральный военно-морской музей,"площадь Труда, 5, Санкт-Петербург, Россия",158858996234,https://navalmuseum.ru/,+7 (812) 303-85-13,Музей,-,"пн,ср,чт,пт,сб,вс 10:00–18:00"
...,...,...,...,...,...,...,...,...,...
495,"[30.388694, 59.712816]",Павильон Концертный зал,"Россия, Санкт-Петербург, Пушкин, Екатеринински...",230115065283,-,-,Музей,-,"ежедневно, 10:00–17:00"
496,"[29.899092, 60.156327]",Башня Шахерезады,"Россия, Санкт-Петербург, Курортный район, посё...",175298957031,-,-,Музей,-,-
497,"[30.275319, 59.881419]",Музей Столярных Инструментов,"ул. Возрождения, 20А, Санкт-Петербург, Россия",1476096141,http://museum.rubankov.ru/,8 (800) 555-55-94,Музей,Выставочный центр,пн-сб 09:00–18:00
498,"[30.593322, 60.010744]",Музейно-мемориальный комплекс Дорога жизни,"Приютинская ул., 13, Всеволожск, Россия",54714480788,http://doroga-zhizni.ru/,+7 (812) 456-11-24,Музей,-,"пн-пт 09:00–17:30, перерыв 13:00–14:00"
